# Importações

In [156]:
import re
import spacy
import pickle
import os
import pandas as pd
import csv
import glob
import csv
from spacy.training import offsets_to_biluo_tags
from spacy.training import biluo_tags_to_spans
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets, biluo_tags_to_spans
import itertools
from sklearn.model_selection import train_test_split
import sys
import matplotlib.pyplot as plt

# importações para o bloco de código do treinamento do modelo de NER
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.tokens import Doc
from spacy.training import Example
import warnings
import datetime as date
from datetime import datetime
import time
from spacy.scorer import Scorer
from sklearn.model_selection import cross_val_score

### Carregando os Dados

In [157]:
#Dados de treinamento rotulado pelo algorimto de rotulação automatica
with open("./DadosResultantesDaRotulacaoAutomatica/Resultado_com_100_porcento.bin", "rb") as arquivo:
    dataSetEntradaTreinamento = pickle.load(arquivo)
len(dataSetEntradaTreinamento)

5855

In [158]:
#Carrega o dataset IOB, ou seja, os tokens são rotulados com tags do tipo marcação IOB.
dataset_teste = pd.read_csv("dataset_rotulado_teste.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim','Documento'])
print("Tamanho do dataset de teste",len(dataset_teste))

Tamanho do dataset de teste 53639


In [159]:
#Carrega o dataset utilizada anteriormente na rotulação manual das categorias de entidades nomeadas, sem os IDs que identificam os documentos
#Esse arquivo é utilizado para extrairmos as setenças a que cada token rotulado no dataset pertence. 
#Essa informação é utilizada no código de conversão do dados no formato de marcação IOB para o formato compreendido pelo spaCy.
with open("dataset_para_rotulacao_manual_versao_2.txt", "r", encoding='utf-8') as file:
    textos = file.read().splitlines()
print("Quantida de de sentenças no dataSet utilizado na rotulação manual:", len(textos))

Quantida de de sentenças no dataSet utilizado na rotulação manual: 13628


### Convertendo os dados do formato IOB para o formato reconhecido pelo spaCy
Dataset de treinamento

In [160]:
sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
s = textos[sMarker]
dataSetEntradaTeste = []
entities = []
for index, row in dataset_teste.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            dataSetEntradaTeste.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]
            
    if(row['Rotulo'][0] == 'O'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if index == dataset_teste.index[-1]: # Ultimo elemento
        if entities:
            dataSetEntradaTeste.append((s, {"entities": [tuple(e) for e in entities]}))
        entities = [] 

print('Quantidade de sentenças do dataset de teste:', len(dataSetEntradaTeste))

Quantidade de sentenças do dataset de teste: 4174


## Importando modelo

In [161]:
# Carregando um modelo do spaCy em portguês previamente treinando
nlp = spacy.load('pt_core_news_sm')
print ("Modelo carregado '% s'"% nlp)

##carregando um modelo em braco
#nlp = spacy.blank('pt')
#nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
#print ("Modelo carregado '% s'"% nlp)
#ner = nlp.create_pipe('ner')
#nlp.add_pipe('ner')

Modelo carregado '<spacy.lang.pt.Portuguese object at 0x000002035D1BEA90>'


## Customizando o Tokenizador para o modelo

In [162]:
from spacy.symbols import ORTH
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import LIST_PUNCT, LIST_ELLIPSES, LIST_QUOTES, LIST_CURRENCY, LIST_HYPHENS
from spacy.lang.char_classes import LIST_ICONS, HYPHENS, CURRENCY
from spacy.lang.char_classes import CONCAT_QUOTES, ALPHA_LOWER, ALPHA_UPPER, ALPHA

# Essa função cria um objeto customizado da classe Tokenizer do spacy, para alinhamento dos tokens com a rotulações em alguns casos especiais por exemplo: '-' e '$'.
def custom_tokenizer(nlp):    
    infixes = (
        spacy.lang.char_classes.LIST_ELLIPSES
        + spacy.lang.char_classes.LIST_ICONS
        + ["\.", "\/", ":", "\(", "\)"]
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[\.])[\.](?=[\.])",
            r"(?<=[{a}{q}])\.(?=[{a}{q}])".format(
                a = ALPHA, q = CONCAT_QUOTES),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}])(?:{h}|\/)(?=[{a}])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[{a}])(?:{h}|\/)(?=[0-9])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/)(?=[{a}])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/|,|\.)(?=[0-9])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}\"]),(?=[0-9{a}])".format(a=ALPHA),
        ]
    )
    list_quotes = ["\\'", '"', '`',
                     '‘', '´', '’', '‚', ',',
                     '„', '»', '«', '「', '」',
                     '『', '』', '（', '）', '〔',
                     '〕', '【', '】', '《', '》',
                     '〈', '〉']
    suffixes = (
        LIST_PUNCT
        + LIST_HYPHENS
        + list_quotes
        + LIST_ICONS
        + [ "\/", ":", "\(", "\)"]
        + ["'s", "'S", "’s", "’S", "—", "–"]
        + [
            r"(?<=[0-9])\+",
            r"(?<=[0-9])(?:{c})".format(c=CURRENCY),
            r"(?<=[0-9a-z(?:{q})])\.".format(
            al=ALPHA_LOWER, q=CONCAT_QUOTES),
            r"(?<=[{au}])\.".format(au=ALPHA_UPPER),
        ] 
    )
    
    prefixes = (
        LIST_PUNCT
        + [ "\/", ":", "\(", "\)"]
        + LIST_HYPHENS
        + list_quotes
        + LIST_CURRENCY
        + LIST_ICONS
    )

    infix_re = spacy.util.compile_infix_regex(infixes)
    suffix_re = spacy.util.compile_suffix_regex(suffixes)
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    custom = Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=None)
    special_case = [{ORTH: "."}, {ORTH: "."}, {ORTH: "."}]        # Adicionando uma regra especial
    custom.add_special_case("...", special_case)
    return custom


In [163]:
# atribuindo o tokenizador customizado ao modelo nlp
nlp.tokenizer = custom_tokenizer(nlp)

### Configurando parâmetros e dados para o treinamento do modelo

In [164]:
# Obtendo o componente do pipeline (ner) para trabalhar com reconhecimentos de entidades nomeadas
ner = nlp.get_pipe('ner')
# Adicionando as categorias de entidade nomeada ao pipeline 'ner' manualmente
ner.add_label('CARACTERISTICA_SENSORIAL_AROMA')
ner.add_label('CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')
ner.add_label('CARACTERISTICA_SENSORIAL_SABOR')
ner.add_label('CARACTERISTICA_SENSORIAL_COR')
ner.add_label('RECIPIENTE_ARMAZENAMENTO')
ner.add_label('EQUIPAMENTO_DESTILACAO')
ner.add_label('CLASSIFICACAO_BEBIDA')
ner.add_label('TEMPO_ARMAZENAMENTO')
ner.add_label('GRADUACAO_ALCOOLICA')
ner.add_label('TIPO_MADEIRA')
ner.add_label('NOME_BEBIDA')
ner.add_label('VOLUME')
ner.add_label('NOME_LOCAL')
ner.add_label('NOME_ORGANIZACAO')
ner.add_label('NOME_PESSOA')
ner.add_label('PRECO')
ner.add_label('TEMPO')

#print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data)
# outra maneira mais rápida de adicionar as categorias de entidades ao 'ner'
#ner = nlp.get_pipe('ner')
#for texto, annotations in TRAIN_DATA:
    #for ent in annotations.get("entities"):
        #ner.add_label(ent[2])     
        #print(ent[2])
#print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data) #verificando as categorias de entidades nomeadas contidas no pipeline

1

In [165]:
# Obtendo os nomes dos componentes que queremos desativar durante o treinamento
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#Abaixo temos outra maneira de fazer a mesma cois, mas com duas linhas de comando
#pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
#unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [168]:
#Tranforma os dados do conjunto de treino no formato entendido pelo modelo ner
examples_train = []
for text, annots in dataSetEntradaTreinamento:
    examples_train.append(Example.from_dict(nlp.make_doc(text), annots))

### Treinando o modelo de reconhecimento de entidades nomeadas
O código abaixo realiza o treinamento do modelo de reconhecimento de entidades nomeadas

In [169]:
#VTreinamento do modelo ner com novas categorias de entidades nomeadas
import decimal
optimizar = nlp.create_optimizer() #quando utilizamos o modelo já treinado do spaCy
#optimizar = nlp.begin_training() #quando é criado um modelo nlp vazio
epochsSize = 50
batchSize = 64
drop_size = 0.30
losses = {}
with nlp.disable_pipes(*unaffected_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy') 
    start_epoch = time.time()
    for epoch in range(epochsSize):
        random.shuffle(examples_train)
        #count = 0
        for batch in minibatch(examples_train, size=batchSize):
            nlp.update(batch, losses=losses, drop=drop_size, sgd=optimizar) 
            and_epoch = time.time()            
            #count+=1
            #valorPerda = losses['ner'] # valorPerda = decimal.Decimal(losses['ner'])
            #print("Perda acumulada ({}) para o batch ({})" .format(valorPerda,count,"\n"))
            tempoParaTreinarModelo = and_epoch-start_epoch 
            #metricas = nlp.evaluate(examples_validation)
            #print('Métricas de validação do modelo para o batch {}'.format(count),'\n')
            #print(metricas,'\n')

        #print("Perda acumulada ({}) para a epóca ({} de {})" .format(losses, epoch + 1, epochsSize),'\n') 
    print("Perda acumulada do modelo ({})" .format(losses),"\n")
    print('Tempo gasto para treinar o modelo em segundos -->', tempoParaTreinarModelo,'\n')
    #metricas=nlp.evaluate(examples_validation)
    #print('Métricas Final de validação do modelo com {} épocas e {} batchs'.format(epochsSize),'\n') 
    #print(metricas,'\n') 
    #scorer = Scorer()
    #result=scorer.score(examples)
    #print('SCORER POR BATC:', result)

Perda acumulada do modelo ({'ner': 112433.80672789947}) 

Tempo gasto para treinar o modelo em segundos --> 1895.9201459884644 



### Comparação Rotulação Manual versus Rotulação do Modelo

In [170]:
#Tranforma os dados do conjunto de teste no formato entendido pelo modelo ner
examples_test = []
for text, annots in dataSetEntradaTeste:
    examples_test.append(Example.from_dict(nlp.make_doc(text), annots))

In [171]:
from collections import Counter
import collections
# Compara item por item como o modelo classificou as sentenças e como elas foram classificadas manual (ponto de referencia).
#nlp = spacy.load(output_dir)
examples = []
comparacao= []
lista_rotulacaomanual = []
lista_rotulacaoautomatica = []
count_1 = 0
count_2 = 0
for text, annots in dataSetEntradaTeste:
    count_1 =count_1+1
    count_2 =count_2+1
    doc = nlp(text)
    anotacao_manual = annots['entities']    
    result_model = [(item.start_char, item.end_char,item.label_) for item in doc.ents]
    #print(f'Classificação Manual:', anotacao_manual,'\n')
    #print(f'Classificação do Modelo:', result_model,'\n')
    
  
    tokens_rotulados_rotulacaomanual = [item for item in anotacao_manual if item[2] != '']
    tokens_rotulados_rotulacaoautomatica = [item for item in result_model if item[2] != '']
    
    if len(tokens_rotulados_rotulacaomanual) < 1:
        tokens_rotulados_rotulacaomanual = [("-","Sem entidade","-")]       
    if len(tokens_rotulados_rotulacaoautomatica) < 1:
        tokens_rotulados_rotulacaoautomatica = [("-","Sem entidade","-")]  
    
    #print("MANUAL")
    #print(tokens_rotulados_rotulacaomanual, count)
    #print("AUTOMATICO")
    #print(tokens_rotulados_rotulacaoautomatica, count)
    
    for i in tokens_rotulados_rotulacaomanual:
        #print(i,";", count_1)  
        lista_rotulacaomanual.append((i,";", count_1))
    for j in tokens_rotulados_rotulacaoautomatica:
        #print(j,";",count_2)  
        lista_rotulacaoautomatica.append((j,";", count_2))
#verifica_listas = bool(set(result_model) & set(tokens_rotulados_rotulacaomanual))

In [172]:
lista_rotulacaomanual_1=[]
for item in lista_rotulacaomanual:
    lista_rotulacaomanual_1.append(item)
len(lista_rotulacaomanual_1)

8104

In [173]:
lista_rotulacaomanual_2=[]
for item in lista_rotulacaoautomatica:
    lista_rotulacaomanual_2.append(item)
len(lista_rotulacaomanual_2)

8929

with open("rotulacaomanual.txt", "w",encoding='utf-8') as arquivo:
for item in lista_rotulacaomanual:
    arquivo.write("%s\n" % str(item))
print('Done')
with open("rotulacaoautomatica.txt", "w",encoding='utf-8') as arquivo:
for item in lista_rotulacaoautomatica:
    arquivo.write("%s\n" % str(item))
print('Done')

### Métricas Precision, Recall e f-measure: 

In [174]:
# Apresenta o resultados das métricas para o modelo como um todo, e as métricas do modelo para cada categoria de entidade
calculatedBySpacy = nlp.evaluate(examples_test)
print(calculatedBySpacy)

#from spacy.scorer import Scorer
#scorer = Scorer()   
#example_scores = []
#for text, annot in dataSetEntradaTeste:
#    print(text)
#    pred = nlp.make_doc(text)
#    temp = Example.from_dict(pred, annot)
#    example_scores.append(temp)
#    scores = scorer.score(example_scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'pos_acc': None, 'morph_acc': None, 'morph_per_feat': None, 'sents_p': 1.0, 'sents_r': 1.0, 'sents_f': 1.0, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': 0.6717351318209688, 'ents_r': 0.7411716953051009, 'ents_f': 0.7047472018525666, 'ents_per_type': {'NOME_BEBIDA': {'p': 0.6666666666666666, 'r': 0.7391304347826086, 'f': 0.7010309278350515}, 'PRECO': {'p': 0.8629629629629629, 'r': 0.8661710037174721, 'f': 0.8645640074211504}, 'VOLUME': {'p': 0.8153034300791556, 'r': 0.807843137254902, 'f': 0.8115561391989493}, 'GRADUACAO_ALCOOLICA': {'p': 0.2777777777777778, 'r': 0.013966480446927373, 'f': 0.02659574468085106}, 'RECIPIENTE_ARMAZENAMENTO': {'p': 0.7196765498652291, 'r': 0.89, 'f': 0.7958271236959761}, 'TIPO_MADEIRA': {'p': 0.7749712973593571, 'r': 0.8302583025830258, 'f': 0.8016627078384799}, 'NOME_LOCAL': {'p': 0.8615622583139985, 'r': 0.8764752163650669, 'f': 0.8689547581903277}, 'NOME_ORGANIZAC

### Avaliação do Modelo pela Métrica Acurácia

In [175]:
# dictionary which will be populated with the entities and result information
entity_evaluation = {}
# helper function to udpate the entity_evaluation dictionary
def update_results(entity, metric):
    if entity not in entity_evaluation:
        entity_evaluation[entity] = {"correct": 0, "total": 0}  
    entity_evaluation[entity][metric] += 1
    
# same as before, see if entities from test set match what spaCy currently predicts
example_scores = []
for data in dataSetEntradaTeste:
    sentence = data[0]
    entities = data[1]["entities"]
    for entity in entities:
        doc = nlp(sentence)
        correct_text = sentence[entity[0]:entity[1]]
        for ent in doc.ents:
            if ent.label_ == entity[2] and ent.text == correct_text:
                update_results(ent.label_, "correct")
                break
        update_results(entity[2], "total")

print("Dados corretamente classificados versus total de dados corretos:",'\n', entity_evaluation)

Dados corretamente classificados versus total de dados corretos: 
 {'': {'correct': 0, 'total': 41711}, 'NOME_BEBIDA': {'correct': 715, 'total': 966}, 'PRECO': {'correct': 233, 'total': 269}, 'VOLUME': {'correct': 618, 'total': 765}, 'GRADUACAO_ALCOOLICA': {'correct': 5, 'total': 358}, 'TIPO_MADEIRA': {'correct': 675, 'total': 813}, 'NOME_ORGANIZACAO': {'correct': 181, 'total': 320}, 'NOME_LOCAL': {'correct': 1115, 'total': 1271}, 'TEMPO_ARMAZENAMENTO': {'correct': 246, 'total': 388}, 'RECIPIENTE_ARMAZENAMENTO': {'correct': 267, 'total': 300}, 'CARACTERISTICA_SENSORIAL_SABOR': {'correct': 139, 'total': 277}, 'CARACTERISTICA_SENSORIAL_COR': {'correct': 111, 'total': 150}, 'TEMPO': {'correct': 343, 'total': 409}, 'CLASSIFICACAO_BEBIDA': {'correct': 363, 'total': 432}, 'CARACTERISTICA_SENSORIAL_AROMA': {'correct': 169, 'total': 281}, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': {'correct': 73, 'total': 85}, 'NOME_PESSOA': {'correct': 170, 'total': 222}, 'EQUIPAMENTO_DESTILACAO': {'correct': 5

In [176]:
print("Calculo Acuracia por Categoria e Acuracia Total")
sum_total = 0
sum_correct = 0
sum_incorretos_total = 0
accuracy_by_category = {}

for entity in entity_evaluation:
    if entity != '':
        total = entity_evaluation[entity]["total"]
        correct = entity_evaluation[entity]["correct"]
        rotulados_incorretamente = total-correct
        sum_total += total
        sum_correct += correct
        sum_incorretos_total += rotulados_incorretamente
        percentual_acertos_modelo_por_categoria = correct/total
        #total_acerto_por_categoria = sum_accuracy_total
        print("{} --> {:.2f}%".format(entity, percentual_acertos_modelo_por_categoria))
        
        accuracy_by_category[entity] = {str(percentual_acertos_modelo_por_categoria)}#para salvar no arquivo .txt com o percentual
sum_accuracy = sum_correct / sum_total 
sum_accuracy_total = str(sum_accuracy) #para salvar no arquivo .txt com o percentual

print("Acuracia Total (todas as categorias): {:.2f}%".format(sum_accuracy))
print('Total de entidades Rotulados Manualmente (referencia):', sum_total)
print('Total de entidades Rotuladas Corretamente pelo modelo:', sum_correct)
print('Total de entidades Rotuladas Incorretamente pelo modelo:', sum_incorretos_total)

Calculo Acuracia por Categoria e Acuracia Total
NOME_BEBIDA --> 0.74%
PRECO --> 0.87%
VOLUME --> 0.81%
GRADUACAO_ALCOOLICA --> 0.01%
TIPO_MADEIRA --> 0.83%
NOME_ORGANIZACAO --> 0.57%
NOME_LOCAL --> 0.88%
TEMPO_ARMAZENAMENTO --> 0.63%
RECIPIENTE_ARMAZENAMENTO --> 0.89%
CARACTERISTICA_SENSORIAL_SABOR --> 0.50%
CARACTERISTICA_SENSORIAL_COR --> 0.74%
TEMPO --> 0.84%
CLASSIFICACAO_BEBIDA --> 0.84%
CARACTERISTICA_SENSORIAL_AROMA --> 0.60%
CARACTERISTICA_SENSORIAL_CONSISTÊNCIA --> 0.86%
NOME_PESSOA --> 0.77%
EQUIPAMENTO_DESTILACAO --> 0.67%
Acuracia Total (todas as categorias): 0.74%
Total de entidades Rotulados Manualmente (referencia): 7391
Total de entidades Rotuladas Corretamente pelo modelo: 5480
Total de entidades Rotuladas Incorretamente pelo modelo: 1911


### Salvados os resultados em um arquivo de texto

In [177]:
#Salvando informações relevantes em um arquivo.txt
nome_arquivo = '100%_metricas.txt'
with open ('./ResultadosComDadosRotuladoAutomaticamente/'+nome_arquivo,"w", encoding='utf-8')  as output:
    quantidadeDados = str(len(dataSetEntradaTeste))
    output.write('Tamanho do dataset de teste: '+quantidadeDados)
    output.write('\n') 
    
    output.write('\n'+"|CONFIGURAÇÕES UTILIZADAS PARA TREINAR O MODELO:|"+'\n')
    epochsSize_String = str(epochsSize)
    batchSize_String = str(batchSize)
    drop_String = drop_size
    output.write('Total de épocas: '+epochsSize_String+"\n")
    output.write('Total de batches: '+batchSize_String+"\n")
    output.write('Tamanho do Drop:')
    output.write(str(drop_size))
    output.write('\n') 
                 
    #Informações sobre treinamento do modelo
    output.writelines('\n'+"|INFORMAÇÕES SOBRE TREINAMENTO DO MODELO:|"+'\n')
    output.writelines('Perda acumulada do modelo (soma de todas as perdas): '+str(losses))
    output.write('\n')
    output.writelines('Tempo gasto para treinar o modelo em segundos: '+str(tempoParaTreinarModelo))
    output.write('\n') 
    
    #Métricas geradas pelo método 'evaluat', disponibilizado pelo próprio spaCy.
    output.writelines('\n'+"|RESULTADOS DO MODELO PARA OS DADOS DO DATASET DE TESTE|")    
    output.writelines('\n'+"|Abaixo temos Precison, Recall, F-measure e Acuracia Geral:|"+'\n')
    ents_p = "Precisão Total: "+str(round(calculatedBySpacy.get("ents_p"),4))+'\n'
    ents_r = "Revocação Total: "+str(round(calculatedBySpacy.get("ents_r"),4))+'\n'
    ents_f = "F-measure Total: "+str(round(calculatedBySpacy.get("ents_f"),4))+'\n'
    ents_a = "Acurácia Total: "+str(round(sum_accuracy,4))+'\n'
    total_dados_com_rotulos = "Total de entidades rotuladas existentes no dataset de teste: "+str(round(sum_total,2))+'\n'
    total_dados_rotulados_corretamento = "Total de entidades classificadas corretamente pelo modelo: "+str(round(sum_correct,2))+'\n'
    total_dados_rotulados_incorretamentos = "Total de entidades classificadas incorretamente pelo modelo: "+str(round(sum_incorretos_total,2))+'\n'
    
    output.write(ents_p),output.write(ents_r),output.write(ents_f),output.write(ents_a)
    output.write(total_dados_com_rotulos),output.write(total_dados_rotulados_corretamento)
    output.write(total_dados_rotulados_incorretamentos)
    #output.writelines(';'.join(str(x) for x in (token_acc,token_p)))
    
    output.writelines('\n'+"|Abaixo temos Precision, Recall e F-measure para cada categoria:|"+'\n')
    ents_per_type = calculatedBySpacy.get("ents_per_type")
    for key in ents_per_type:
        ents_por_tipo = str(key)+str(ents_per_type[key])+'\n'
        output.write(ents_por_tipo) 
   
    #Acuracia do modelo por categoria.
    output.writelines('\n'+"|Abaixo temos a Acurácia para cada categoria:|"+'\n')
    for key in accuracy_by_category:
        entity_evaluation_accuracy = str(key)+str(accuracy_by_category[key])+'\n'
        output.write(entity_evaluation_accuracy)
    
    #Total de acertos e erros do modelo por categoria.    
    output.writelines('\n'+"|Abaixo temos o total de acertos do modelo para cada categoria Versus o total que ele deveria classificar:|"+'\n')
    for key in entity_evaluation:
        if key != '':
            acertos_erros = str(key)+str(entity_evaluation[key])+'\n'
            output.write(acertos_erros) 
            
    #Parametros digitados no terminal pelo usuario
    #output.write('\n'+"|ABAIXO TEMOS OS PARAMETROS INFORMADOS NO TEMRINAL:|"+'\n')
    #output.writelines(';'.join(str(x) for x in (primeiraEntradaTerminal,segundaEntradaTerminal,terceiraEntradaTerminal,quartaEntradaTerminal,quintaEntradaTerminal)))

In [178]:
#Dados Rotulados manualmente.

nome_arquivo_2 = '100%_rotulacaomanual.txt'
with open ('./ResultadosComDadosRotuladoManualmente/'+nome_arquivo_2,"w", encoding='utf-8')  as output:
    for key in lista_rotulacaomanual_1:
        key = str(key)+'\n'
        output.write(key)

#Dados Rotulados atomaticamente.

nome_arquivo_3 = '100%_rotulacaoautomatical.txt'
with open ('./ResultadosComDadosRotuladoAutomaticamente/'+nome_arquivo_3,"w", encoding='utf-8')  as output:
    for key in lista_rotulacaomanual_2:
        key = str(key)+'\n'
        output.write(key)